MAI 2025 GAN Deepfake Project

Задача с deepfake показалась максимально захватывающей, вызывающей и сложной, которая должна пополнить копилку опыта и навыков.

Было принято решение перенять опыт - опробовать в деле текущие популярные, где-то даже "нашумевшие" открытые репозитории предлагающие deepfake функционал. 

DeepFaceLab - самое популярное, интересное решение, но уже не поддерживается, последнее обновление датировано ноябрем 2024. Код стройный, но запустить так и не удалось. Пазл не собрался.

FaceSwap - имеет относительно свежие обновления, но как показало знакомство рабочим по умолчанию не является, проблемы виртуальных окружений и совместимости пакетов python не дали далко продвинуться.
Был даже найден установщик последнего релиза под Windows, но и его работоспособность напрямую зависит от версии установленного Python, его пакетов библиотек, которых уже нет в репозиториях.

SimSwap - показался самым запутанным, и самым не подходящим по организации взаимодействия, но это единственное решение, которое смогло показать результат, В релизных сборках явно присутствуют деффекты, наример как ссылки на локальные диски, а организация кода оставляет желать лучшего. С большими усиллиями и дебагингом всё же удалось получить хоть какой-то результат, но об 
подробне немного позже.

First Order Motion Mode - последнее обновление датировано ноябрем 2024. Самое скромное решение из популярных. Минимум документации, максимум боли, и нулевой результат. Все попытки реанимитровать его потерпели неудачу. 

Roop - по заявлениям разработчика, это всё ещё рабочий инструмент, но он тоже заброшен в марте 2024. И тоже потребовал уникального набора библиотек, которые просто конфилоиктовали друг с другом по кругу.

Решения использующие Python 3.6 и ниже, сразу отсекались, так как только на установку старой версии Pyhton можно потратить всё отведенное на проект время.

Первое с чем сталлкивается исследователь это боль работы с GPU. Вроде уже все понимают, что без GPU в этом процессе никуда, и есть две основные бибилиотеки tensorflow и torch, которые имеют сбори использующие графический ускоритель.

Установив их посделние версии, и опробовав использовать графические ядра мы сталкиваемся с данностью - CUDA ядра не видны и не применяются библиотеками.
Изучив внимательно примечания на сайте tensorflow, была получена информация о том, что сборка использующая GPU доступна только в Linux или WSL2.

Выводы сделаны, мы получили прямое указание работать в WSL, это с недавних пор стало удобно выполнять из VS Code. 
Запускаем WSL ставим Ubuntu. Ставим драйвера nvidia, накатываем torch и tensorflow. Регистрируем разые python окружения. И только это хоть как-то спасает ситуацию.


Забегая немного вперёд, скажу, что работая над проектом получилось максимально собрать незабываемый и непередаваемый опыт по получению доступа к своему оборудованию под нужды исследования.

**Глава 1. Что получилось - результат 1. SimSwap**

Из всего многообразия исследованных инструментов, на актуальном стеке Python >=3.12 удалось запустить только SimSwap.
Не без трудностей, не без дебага, с исправлением конфликтов бибилиотек, с поиском предварительно отренированных весов.
Удалось получить скромный результат по замене лица в видео файле. Исследованы и взяты на вооружение подходы к обработке видео кадров через ffmpg.

Пример полученного результата размещен в каталоге data/results/simswap

Попытки дообучить модель на своём датасете потерпели крах, как и попытки обучить модель с нуля на том же датасете.

О промышленном или публичном применении полученной конфигурации говорить не приходится. Единственный относительно рабочий вариант - собрать докер образ, но это не цель нашего проекта.

В попытках хоть как-то приблизиться к системе создания deepfakе,было принято следующее решение - попробовать реализовать модель для проекта самостоятельно.

Первым шагом намечено получение устойчивого результата по замене лиц. Опираясь на SimSwap попытаться собрать приемлемый результат.

Получив устойчивый резульатат для одного изображения, можно будет перейти к работе с видео.

Перавым делом был собран датасет из личного альбома. 
Получены 387 реальных изображений одного лица, в разных ракурсах и в разной сложности узнаваимости. 256x256 и 512х512 в каталоге data/train/

Импортируем необходимые модули.

In [52]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import os

from torchvision import transforms
from torch.utils.data import DataLoader

Пробуем с малого. Небольшой энкодер в 4 слоя и RELU между ними

In [ ]:
class IDEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            
            nn.Conv2d(3, 64, 5, stride=2, padding=2),  # 256x256 → 128x128
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 5, stride=2, padding=2), # 128x128 → 64x64
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, 5, stride=2, padding=2), # 64x64 → 32x32
            nn.InstanceNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)  # Глобальный пулинг
        )
        self.fc = nn.Linear(256, 512)  # ID-вектор

    def forward(self, x):
        x = self.layers(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

Соответствующий энкодеру декодер.

In [55]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(256 + 512, 256, 3, padding=1),  # Объединяем features + ID
            nn.InstanceNorm2d(256),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),  # 32x32 → 64x64
            nn.Conv2d(256, 128, 3, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),  # 64x64 → 128x128
            nn.Conv2d(128, 64, 3, padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),  # 128x128 → 256x256
            nn.Conv2d(64, 3, 3, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.layers(x)

In [54]:
class Swapper(nn.Module):
    def __init__(self, latent_dim=512):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(latent_dim * 2, latent_dim),
            nn.ReLU(),
            nn.Linear(latent_dim, latent_dim)
        )

    def forward(self, source_id, target_features):
        # source_id: [B, 512], target_features: [B, C, H, W]
        B, C, H, W = target_features.shape
        id_expanded = source_id.view(B, -1, 1, 1).expand(-1, -1, H, W)
        return torch.cat([target_features, id_expanded], dim=1)

Ну и собственно сама магия. Извлечение вектора, извлечение фич, объединение и отправка в декодер

In [56]:
class SimpleSwap(nn.Module):
    def __init__(self):
        super().__init__()
        self.id_encoder = IDEncoder()
        self.feature_extractor = IDEncoder()  # Для извлечения признаков позы/мимики
        self.swapper = Swapper()
        self.decoder = Decoder()

    def forward(self, source_img, target_img):
        # Извлекаем ID исходного лица
        source_id = self.id_encoder(source_img)
        
        # Извлекаем признаки позы/мимики из целевого изображения
        target_features = self.feature_extractor.layers[:-1](target_img)  # Без FC-слоя
        
        # Объединяем ID и признаки
        swapped = self.swapper(source_id, target_features)
        
        # Генерируем результат
        return self.decoder(swapped)

Определяем функцию потерь для предстоящего обучения.

In [57]:
class LossCalculator:
    def __init__(self, device):
        self.mse = nn.MSELoss()
        self.vgg = models.vgg16(pretrained=True).features[:16].to(device).eval()
        for param in self.vgg.parameters():
            param.requires_grad = False

    def perceptual_loss(self, x, y):
        return torch.mean(torch.abs(self.vgg(x) - self.vgg(y)))

    def compute(self, real, fake, swapped):
        # Reconstruction loss
        loss_rec = self.mse(real, fake)
        
        # Perceptual loss
        loss_perc = self.perceptual_loss(real, fake)
        
        # Adversarial loss (опционально)
        # Можно добавить GAN-компонент
        
        return 0.7 * loss_rec + 0.3 * loss_perc

Декларация процесса обучения

In [58]:
def train(model, dataloader, epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0002, betas=(0.5, 0.999))
    criterion = LossCalculator(device)

    for epoch in range(epochs):
        for source, target in dataloader:
            source, target = source.to(device), target.to(device)
            
            # Генерация
            swapped = model(source, target)
            
            # Вычисление потерь
            loss = criterion.compute(target, swapped, source)
            
            # Обновление весов
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

немного удобств для последующего применения.

In [59]:
def swap_face(model, source_img, target_frame):
    model.eval()
    with torch.no_grad():
        swapped = model(source_img, target_frame)
    return swapped


In [ ]:
def load_image(image_path, img_size=256, device="cuda"):
    """
    Загружает изображение, обнаруживает лицо, выравнивает и нормализует.
    Возвращает тензор [1, 3, H, W] в диапазоне [-1, 1].
    """
    # Загрузка изображения и конвертация в RGB
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Обнаружение лица (упрощенная версия - для реального использования лучше MTCNN)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    if len(faces) == 0:
        raise ValueError("Лицо не обнаружено на изображении!")
    
    # Вырезаем лицо
    (x, y, w, h) = faces[0]
    face = image[y:y+h, x:x+w]
    
    # Ресайз и нормализация
    face = cv2.resize(face, (img_size, img_size))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    face_tensor = transform(face).unsqueeze(0).to(device)
    
    return face_tensor

def save_image(tensor, path):
    """Сохраняет тензор [1, 3, H, W] в диапазоне [-1, 1] как изображение"""
    tensor = tensor.squeeze(0).cpu().detach()
    tensor = tensor * 0.5 + 0.5  # [-1, 1] -> [0, 1]
    transforms.ToPILImage()(tensor).save(path)

Класс обертка для датасета с лицами

In [ ]:
class FaceDataset(torch.utils.data.Dataset):
    def __init__(self, source_dir, target_dir, img_size=256):
        self.source_paths = sorted([os.path.join(source_dir, f) for f in os.listdir(source_dir)])
        self.target_paths = sorted([os.path.join(target_dir, f) for f in os.listdir(target_dir)])
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        self.img_size = img_size

    def __len__(self):
        return min(len(self.source_paths), len(self.target_paths))

    def __getitem__(self, idx):
        src_img = self._load_image(self.source_paths[idx])
        tgt_img = self._load_image(self.target_paths[idx])
        return src_img, tgt_img

    def _load_image(self, path):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.img_size, self.img_size))
        return self.transform(img)

Создаем dataset

In [62]:
dataset = FaceDataset("data/train/source", "data/train/target_256")

Запуск процесса обучения.

In [63]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
model = SimpleSwap().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

In [64]:
for epoch in range(100):
    for batch_src, batch_tgt in dataloader:
        batch_src, batch_tgt = batch_src.cuda(), batch_tgt.cuda()
        
        # Forward pass
        swapped = model(batch_src, batch_tgt)
        
        # Loss calculation (например, MSE + Perceptual Loss)
        loss = torch.nn.functional.mse_loss(swapped, batch_tgt)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
    torch.save(model.state_dict(), f"data/train/weights/simpleswap_99_{epoch}.pth")  # Сохраняем веса

Epoch 0, Loss: 0.0443
Epoch 1, Loss: 0.0275
Epoch 2, Loss: 0.0355
Epoch 3, Loss: 0.0272
Epoch 4, Loss: 0.0302
Epoch 5, Loss: 0.0131
Epoch 6, Loss: 0.0373
Epoch 7, Loss: 0.0187
Epoch 8, Loss: 0.0328
Epoch 9, Loss: 0.0172
Epoch 10, Loss: 0.0220
Epoch 11, Loss: 0.0190
Epoch 12, Loss: 0.0146
Epoch 13, Loss: 0.0107
Epoch 14, Loss: 0.0123
Epoch 15, Loss: 0.0182
Epoch 16, Loss: 0.0097
Epoch 17, Loss: 0.0249
Epoch 18, Loss: 0.0112
Epoch 19, Loss: 0.0103
Epoch 20, Loss: 0.0152
Epoch 21, Loss: 0.0081
Epoch 22, Loss: 0.0092
Epoch 23, Loss: 0.0153
Epoch 24, Loss: 0.0087
Epoch 25, Loss: 0.0120
Epoch 26, Loss: 0.0091
Epoch 27, Loss: 0.0070
Epoch 28, Loss: 0.0137
Epoch 29, Loss: 0.0077
Epoch 30, Loss: 0.0118
Epoch 31, Loss: 0.0100
Epoch 32, Loss: 0.0076
Epoch 33, Loss: 0.0263
Epoch 34, Loss: 0.0058
Epoch 35, Loss: 0.0082
Epoch 36, Loss: 0.0429
Epoch 37, Loss: 0.0068
Epoch 38, Loss: 0.0069
Epoch 39, Loss: 0.0080
Epoch 40, Loss: 0.0047
Epoch 41, Loss: 0.0054
Epoch 42, Loss: 0.0058
Epoch 43, Loss: 0.006

KeyboardInterrupt: 

In [ ]:
# когда надо сохранить какой-либо набор значений весов
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': 99,
    'loss': 0.0059,
}, "simpleswap_99_fix.pth")

In [ ]:
# когда надо загрузить в модель какой-либо набор значений весов
checkpoint = torch.load("simpleswap_99.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

In [ ]:
# посмотреть модель
model.eval()

SimpleSwap(
  (id_encoder): IDEncoder(
    (layers): Sequential(
      (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): ReLU()
      (3): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (4): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (5): ReLU()
      (6): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (7): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (8): ReLU()
      (9): AdaptiveAvgPool2d(output_size=1)
    )
    (fc): Linear(in_features=256, out_features=512, bias=True)
  )
  (feature_extractor): IDEncoder(
    (layers): Sequential(
      (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2)

In [65]:
src = "data/src/lic.jpg"
trg = "data/src/target.jpg"
res = "data/result/r_03.jpg"
#wgh = "/mnt/c/code/GENS/lica/data/weights/latest_net_G.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = SimpleSwap().to(device)
# model.load_state_dict(torch.load(wgh))

# # Загрузка изображений
source_img = load_image(src, device=device)
target_frame = load_image(trg, device=device)

# Замена лица
result = swap_face(model, source_img, target_frame)

# Сохранение результата
save_image(result, res)


In [66]:
src = "data/src/lic.jpg"
trg = "data/src/target.jpg"
source_img = load_image(src, device=device)
target_frame = load_image(trg, device=device)
res = "data/result/r_032.jpg"
with torch.no_grad():
    swapped = model(source_img, target_frame)
    save_image(swapped, res)

**Вывод**

Портить видео и фотографии мы научились, осталось дело за малым - достичь совершенства.
Показывать данное решение на общую публику не рекомендовано.

Следующий шаг - попытка применить GAN хотя бы на уровне порчи фортографий.